In [4]:
import glob
import re
import warnings
import numpy as np
import csv
from itertools import combinations
from math import acos, degrees
from ase.io import read
from scipy.spatial import ConvexHull

# ============================================================
# Silence harmless ASE CIF warnings
# ============================================================

warnings.filterwarnings(
    "ignore",
    message="crystal system .* is not interpreted"
)

# ============================================================
# Geometry utilities
# ============================================================

def angle(v1, v2):
    """Angle (deg) between two vectors"""
    cosang = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    cosang = np.clip(cosang, -1.0, 1.0)
    return degrees(acos(cosang))

# ============================================================
# Core FeN6 analysis (Collet–Guionneau)
# ============================================================

def analyze_fe_site(atoms, fe_index):
    """
    FeN6 distortion parameters:
    <Fe–N>, zeta, Sigma, Theta (angular trigonal), Vp
    """

    fe_pos = atoms[fe_index].position

    # --------------------------------------------------------
    # Collect Fe–N vectors
    # --------------------------------------------------------
    lig = []
    for i, atom in enumerate(atoms):
        if atom.symbol == "N":
            vec = atoms.get_distance(fe_index, i, mic=True, vector=True)
            dist = np.linalg.norm(vec)
            lig.append((dist, vec))

    if len(lig) < 6:
        raise RuntimeError(f"Fe index {fe_index}: fewer than 6 N atoms found")

    # --------------------------------------------------------
    # Select 6 nearest N atoms
    # --------------------------------------------------------
    lig.sort(key=lambda x: x[0])
    six = lig[:6]

    bond_lengths = np.array([d for d, _ in six])
    n_pos = np.array([fe_pos + vec for _, vec in six])

    # --------------------------------------------------------
    # 1. Average Fe–N
    # --------------------------------------------------------
    avg_bond = bond_lengths.mean()

    # --------------------------------------------------------
    # 2. Bond-length distortion ζ
    # --------------------------------------------------------
    zeta = bond_lengths.max() - bond_lengths.min()

    # --------------------------------------------------------
    # 3. Angular distortion Σ
    # --------------------------------------------------------
    angles = []
    for i, j in combinations(range(6), 2):
        v1 = n_pos[i] - fe_pos
        v2 = n_pos[j] - fe_pos
        angles.append(angle(v1, v2))

    angles.sort()
    cis_angles = angles[:12]  # remove 3 trans angles
    Sigma = sum(abs(a - 90.0) for a in cis_angles)

    # --------------------------------------------------------
    # 4. Trigonal angular distortion Θ
    #    (robust face-based definition)
    # --------------------------------------------------------
    u = []
    for pos in n_pos:
        v = pos - fe_pos
        u.append(v / np.linalg.norm(v))
    u = np.array(u)

    face_alphas = []

    for i, j, k in combinations(range(6), 3):

        a_ij = angle(u[i], u[j])
        a_jk = angle(u[j], u[k])
        a_ki = angle(u[k], u[i])

        # trigonal faces = all cis
        if (
            70.0 < a_ij < 110.0 and
            70.0 < a_jk < 110.0 and
            70.0 < a_ki < 110.0
        ):
            alpha = 180.0 - 0.5 * (a_ij + a_jk + a_ki)
            face_alphas.append(alpha)

    #if len(face_alphas) < 4:
        #print(
            #f"⚠️ Warning: Fe index {fe_index} – "
            #f"only {len(face_alphas)} trigonal faces detected"
        #)

    Theta = np.mean([abs(a - 60.0) for a in face_alphas])

    # --------------------------------------------------------
    # 5. Polyhedral volume Vp
    # --------------------------------------------------------
    hull = ConvexHull(n_pos)
    Vp = hull.volume

    return avg_bond, zeta, Sigma, Theta, Vp

# ============================================================
# Batch processing + table output
# ============================================================

results = []

print(
    f"{'CIF':25s} {'T(K)':>6s} {'Fe':>4s} "
    f"{'<Fe–N>(Å)':>12s} {'ζ(Å)':>8s} "
    f"{'Σ(deg)':>10s} {'Θ(deg)':>10s} {'Vp(Å³)':>10s}"
)
print("-" * 95)

for cif in sorted(glob.glob("*.cif")):

    atoms = read(cif)

    temp_match = re.search(r"(\d+)\s*[Kk]", cif)
    temperature = int(temp_match.group(1)) if temp_match else None

    for i, atom in enumerate(atoms):
        if atom.symbol == "Fe":

            avg_bond, zeta, Sigma, Theta, Vp = analyze_fe_site(atoms, i)

            print(
                f"{cif:25s} "
                f"{str(temperature) if temperature else 'NA':>6s} "
                f"{i:4d} "
                f"{avg_bond:12.4f} "
                f"{zeta:8.4f} "
                f"{Sigma:10.2f} "
                f"{Theta:10.3f} "
                f"{Vp:10.3f}"
            )

            results.append([
                cif,
                temperature,
                i,
                avg_bond,
                zeta,
                Sigma,
                Theta,
                Vp
            ])

# ============================================================
# Save CSV
# ============================================================

with open("FeN6_ColletGuionneau_distortion_vs_T.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "CIF",
        "Temperature (K)",
        "Fe_index",
        "<Fe–N> (Å)",
        "zeta (Å)",
        "Sigma (deg)",
        "Theta (deg)",
        "Vp (Å^3)"
    ])
    writer.writerows(results)

print("\n✅ Analysis complete → FeN6_ColletGuionneau_distortion_vs_T.csv")


CIF                         T(K)   Fe    <Fe–N>(Å)     ζ(Å)     Σ(deg)     Θ(deg)     Vp(Å³)
-----------------------------------------------------------------------------------------------
M1_100k - Copy.cif           100    0       2.0490   0.5613     151.95      5.781     10.282
M1_100k - Copy.cif           100    1       2.0490   0.5613     151.95      5.781     10.282
M1_100k - Copy.cif           100    2       2.0490   0.5613     151.95      5.781     10.282
M1_100k - Copy.cif           100    3       2.0490   0.5613     151.95      5.781     10.282
M1_150K - Copy.cif           150    0       2.1236   0.2733     192.85     11.231     10.701
M1_150K - Copy.cif           150    1       2.1236   0.2733     192.85     11.231     10.701
M1_150K - Copy.cif           150    2       2.1236   0.2733     192.85     11.231     10.701
M1_150K - Copy.cif           150    3       2.1236   0.2733     192.85     11.231     10.701
M1_175K - Copy.cif           175    0       2.1199   0.2601     188

/home/krishna_msc/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/krishna_msc/.local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [2]:
#!/usr/bin/env python3
"""
rmsd_asu_match.py

Reads two CIFs (HS and LS), extracts the asymmetric unit (unique positions
from the CIF), matches atoms by element with an optimal assignment,
applies periodic minimum-image convention, aligns HS->LS (Kabsch),
computes per-atom displacements, prints Average RMSD and Max deviation,
writes per-atom CSV/text outputs and a combined CIF for VESTA with labels.

Requirements:
  - ase
  - numpy
  - scipy (optional but recommended; if missing the script falls back to greedy matching)

Usage:
  - edit the hs_file and ls_file below or pass them via command line (quick edit)
  - python rmsd_asu_match.py
"""

import numpy as np
from ase.io import read, write
import math
import sys
import os

# Try to import Hungarian assignment
try:
    from scipy.optimize import linear_sum_assignment
    SCIPY_AVAILABLE = True
except Exception:
    SCIPY_AVAILABLE = False

# ----------------- USER INPUT -----------------
hs_file = "M2_300K - Copy.cif"   # high-spin CIF
ls_file = "M2_100K - Copy.cif"          # low-spin CIF
use_only_heavy_atoms = False     # ignore H? (True/False)
out_prefix = "rmsd_out"          # output files prefix
uniqueness_tol = 1e-3           # tolerance in fractional coords to deduplicate
# ------------------------------------------------

def get_asymmetric_unit(atoms, tol=1e-3, keep_H=True):
    """
    Return (symbols_frac, frac_positions, cart_positions, original_indices)
    representing a deduplicated (unique) set of atomic sites from the CIF by
    grouping positions that are identical modulo lattice translations.
    This approximates the asymmetric unit if the CIF contains symmetry-expanded atoms.
    """
    frac = atoms.get_scaled_positions(wrap=True)
    cart = atoms.get_positions()
    syms = np.array(atoms.get_chemical_symbols())
    keep_mask = np.ones(len(syms), dtype=bool)
    if not keep_H:
        keep_mask = (atoms.get_atomic_numbers() > 1)
    frac = frac[keep_mask]
    cart = cart[keep_mask]
    syms = syms[keep_mask]

    unique_syms = []
    unique_frac = []
    unique_cart = []
    unique_orig_idx = []

    def frac_dist(a, b):
        # minimum-image distance in fractional coordinates
        d = a - b
        d = d - np.round(d)  # put into -0.5..0.5
        return np.linalg.norm(d)

    for i, (f, c, s) in enumerate(zip(frac, cart, syms)):
        found = False
        for j, uf in enumerate(unique_frac):
            if s != unique_syms[j]:
                # only combine same element; different elements can have very close coords in some cases
                continue
            if frac_dist(f, uf) < tol:
                found = True
                break
        if not found:
            unique_syms.append(s)
            unique_frac.append(f.copy())
            unique_cart.append(c.copy())
            unique_orig_idx.append(i)
    # convert lists to arrays
    unique_frac = np.array(unique_frac)
    unique_cart = np.array(unique_cart)
    unique_syms = np.array(unique_syms)
    return unique_syms, unique_frac, unique_cart, unique_orig_idx

def frac_to_cart(frac, cell):
    """Convert Nx3 fractional coords to Cartesian using cell (3x3 array)."""
    # cart = frac @ cell
    return np.dot(frac, cell)

def minimum_image_frac_delta(p_frac, q_frac):
    """Return fractional delta vector with minimum image (range -0.5..+0.5)."""
    d = p_frac - q_frac
    d = d - np.round(d)
    return d

def kabsch(P, Q):
    """Return rotation matrix R that aligns P -> Q (both centered)."""
    H = P.T @ Q
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt.T @ U.T
    return R

def match_atoms_by_element_and_distance(symA, fracA, symB, fracB, cell, use_scipy=True):
    """
    Return index arrays (idxA, idxB) such that symA[idxA[k]] matches symB[idxB[k]].
    Matching is per-element and solves minimal total Cartesian distances using Hungarian algorithm.
    Fallback: greedy nearest-neighbour per element if scipy not available.
    """
    if len(symA) != len(symB):
        raise ValueError("Different numbers of unique atoms: {} vs {}".format(len(symA), len(symB)))

    N = len(symA)
    idxA_final = []
    idxB_final = []

    elements = sorted(set(symA))
    for el in elements:
        # indices of element el in both lists
        ia = [i for i,s in enumerate(symA) if s == el]
        ib = [i for i,s in enumerate(symB) if s == el]
        if len(ia) != len(ib):
            # can't do perfect element-wise matching -> try to proceed but warn
            print(f"Warning: element counts differ for {el}: {len(ia)} vs {len(ib)}. Will attempt global matching.")
            # fallback to global matching
            return match_atoms_globally(symA, fracA, symB, fracB, cell, use_scipy)
        na = len(ia)
        if na == 0:
            continue
        # build cost matrix (Cartesian distances using minimum-image)
        cost = np.zeros((na, na))
        for ii, aidx in enumerate(ia):
            for jj, bidx in enumerate(ib):
                df = minimum_image_frac_delta(fracA[aidx], fracB[bidx])
                dcart = frac_to_cart(df, cell)
                cost[ii, jj] = np.linalg.norm(dcart)
        if use_scipy and SCIPY_AVAILABLE:
            row, col = linear_sum_assignment(cost)
        else:
            # greedy fallback: assign each row to nearest unassigned column
            row = []
            col = []
            available = set(range(na))
            for ii in range(na):
                # find nearest available column
                nearest_j = min(available, key=lambda j: cost[ii, j])
                row.append(ii)
                col.append(nearest_j)
                available.remove(nearest_j)
        # map back to original indices
        for r, c in zip(row, col):
            idxA_final.append(ia[r])
            idxB_final.append(ib[c])
    # Return index arrays in order of idxA_final
    # sort by idxA_final to have stable ordering
    order = np.argsort(idxA_final)
    idxA_final = [idxA_final[i] for i in order]
    idxB_final = [idxB_final[i] for i in order]
    return np.array(idxA_final, dtype=int), np.array(idxB_final, dtype=int)

def match_atoms_globally(symA, fracA, symB, fracB, cell, use_scipy=True):
    """
    If per-element counts don't match, do a global assignment minimizing distances
    but penalize element mismatch heavily.
    """
    N = len(symA)
    cost = np.zeros((N, N))
    big_penalty = 1e3
    for i in range(N):
        for j in range(N):
            el_pen = 0 if symA[i] == symB[j] else big_penalty
            df = minimum_image_frac_delta(fracA[i], fracB[j])
            dcart = frac_to_cart(df, cell)
            cost[i, j] = np.linalg.norm(dcart) + el_pen
    if use_scipy and SCIPY_AVAILABLE:
        row, col = linear_sum_assignment(cost)
    else:
        # greedy fallback
        row = []
        col = []
        available = set(range(N))
        for i in range(N):
            nearest_j = min(available, key=lambda j: cost[i, j])
            row.append(i)
            col.append(nearest_j)
            available.remove(nearest_j)
    order = np.argsort(row)
    return np.array(row)[order], np.array(col)[order]

def main():
    # 1) Read CIFs
    if not os.path.exists(hs_file):
        print("HS file not found:", hs_file); sys.exit(1)
    if not os.path.exists(ls_file):
        print("LS file not found:", ls_file); sys.exit(1)

    hs_atoms = read(hs_file)
    ls_atoms = read(ls_file)

    if use_only_heavy_atoms:
        print("NOTE: hydrogens will be excluded from asymmetric-unit extraction and matching.")

    # 2) Extract asymmetric unit candidates
    syms_hs, frac_hs, cart_hs, idx_hs = get_asymmetric_unit(hs_atoms, tol=uniqueness_tol, keep_H=not use_only_heavy_atoms)
    syms_ls, frac_ls, cart_ls, idx_ls = get_asymmetric_unit(ls_atoms, tol=uniqueness_tol, keep_H=not use_only_heavy_atoms)

    print(f"Unique sites found: HS={len(syms_hs)}  LS={len(syms_ls)}")
    if len(syms_hs) != len(syms_ls):
        print("Warning: number of unique sites differs. Will still attempt global matching.")

    # Use LS cell for Cartesian conversions & as reference
    cell = np.array(ls_atoms.get_cell())
    # 3) Match atoms (indices arrays)
    try:
        ia, ib = match_atoms_by_element_and_distance(syms_hs, frac_hs, syms_ls, frac_ls, cell, use_scipy=True)
    except Exception as e:
        print("Error during per-element matching:", e)
        print("Falling back to global matching.")
        ia, ib = match_atoms_globally(syms_hs, frac_hs, syms_ls, frac_ls, cell, use_scipy=True)

    # reorder arrays to matched order
    syms_hs_matched = syms_hs[ia]
    syms_ls_matched = syms_ls[ib]
    frac_hs_matched = frac_hs[ia]
    frac_ls_matched = frac_ls[ib]

    # convert fractional to Cartesian in LS cell (apply minimum-image for matches)
    cart_ls_matched = frac_to_cart(frac_ls_matched, cell)
    # For HS we compute delta fractional (min image) relative to matched LS and convert to cart
    cart_hs_matched = np.zeros_like(cart_ls_matched)
    for k in range(len(frac_hs_matched)):
        df = minimum_image_frac_delta(frac_hs_matched[k], frac_ls_matched[k])
        cart_hs_matched[k] = frac_to_cart(frac_ls_matched[k] + df, cell)  # effectively aligned in same cell

    # 4) Center and apply Kabsch: align HS -> LS
    P = cart_hs_matched.copy()
    Q = cart_ls_matched.copy()
    P_cent = P - P.mean(axis=0)
    Q_cent = Q - Q.mean(axis=0)
    R = kabsch(P_cent, Q_cent)
    P_aligned = (P_cent @ R)  # still centered like Q_cent
    # position HS aligned into LS frame by shifting to Q mean:
    P_aligned_cart = P_aligned + Q.mean(axis=0)

    # compute per-atom deviation vectors and distances
    dvecs = P_aligned_cart - Q
    dists = np.linalg.norm(dvecs, axis=1)
    avg_rmsd = math.sqrt(np.mean(dists**2))
    max_dev = float(np.max(dists))
    max_idx = int(np.argmax(dists))  # index in matched arrays
    print("\n=== RESULTS ===")
    print(f"Number of atoms compared: {len(dists)}")
    print(f"Average RMSD (Å): {avg_rmsd:.4f}")
    print(f"Maximum deviation (Å): {max_dev:.4f}  (matched atom index: {max_idx}, element: {syms_hs_matched[max_idx]})")

    # 5) Write outputs
    txt_out = out_prefix + "_per_atom.txt"
    csv_out = out_prefix + "_per_atom.csv"
    cif_out = out_prefix + "_overlay.cif"

    with open(txt_out, "w") as f:
        f.write("# idx  elem  HS_frac_x HS_frac_y HS_frac_z   HS_cart_x HS_cart_y HS_cart_z   "
                "LS_frac_x LS_frac_y LS_frac_z   LS_cart_x LS_cart_y LS_cart_z   disp_A\n")
        for i, (s, hf, hc, lf, lc, dist) in enumerate(zip(syms_hs_matched, frac_hs_matched, P_aligned_cart,
                                                        frac_ls_matched, cart_ls_matched, dists), start=1):
            f.write(f"{i:3d} {s:>2s}  "
                    f"{hf[0]:8.5f} {hf[1]:8.5f} {hf[2]:8.5f}  "
                    f"{hc[0]:10.5f} {hc[1]:10.5f} {hc[2]:10.5f}  "
                    f"{lf[0]:8.5f} {lf[1]:8.5f} {lf[2]:8.5f}  "
                    f"{lc[0]:10.5f} {lc[1]:10.5f} {lc[2]:10.5f}  "
                    f"{dist:8.5f}\n")
    # CSV
    with open(csv_out, "w") as f:
        f.write("idx,elem,HS_frac_x,HS_frac_y,HS_frac_z,HS_cart_x,HS_cart_y,HS_cart_z,"
                "LS_frac_x,LS_frac_y,LS_frac_z,LS_cart_x,LS_cart_y,LS_cart_z,disp_A\n")
        for i, (s, hf, hc, lf, lc, dist) in enumerate(zip(syms_hs_matched, frac_hs_matched, P_aligned_cart,
                                                        frac_ls_matched, cart_ls_matched, dists), start=1):
            f.write(f"{i},{s},{hf[0]:.6f},{hf[1]:.6f},{hf[2]:.6f},"
                    f"{hc[0]:.6f},{hc[1]:.6f},{hc[2]:.6f},"
                    f"{lf[0]:.6f},{lf[1]:.6f},{lf[2]:.6f},"
                    f"{lc[0]:.6f},{lc[1]:.6f},{lc[2]:.6f},{dist:.6f}\n")

    print(f"Wrote per-atom text -> {txt_out}")
    print(f"Wrote per-atom CSV  -> {csv_out}")

    # 6) Create overlay CIF for VESTA:
    # We'll create a combined Atoms object: LS atoms (normal labels) + HS aligned atoms (labels include _HS, and the max gets _MAX)
    # Use original LS atoms positions for LS block, and use P_aligned_cart for HS block.
    ls_copy = ls_atoms.copy()
    hs_copy = ls_atoms.copy()  # create a copy to get right cell / arrays; we'll overwrite positions & symbols for HS block

    # We'll build new atoms object manually using ase.Atoms concatenation
    from ase import Atoms
    # Get LS atoms in the same order as matched list (ls matched indices ib)
    # But we earlier had idx_ls original mapping; ib indices refer to unique list indices.
    # We'll extract positions and symbols for the matched LS unique sites.
    ls_syms_matched = syms_ls_matched
    ls_cart_matched = cart_ls_matched

    # Build LS atoms (ordered)
    atoms_ls_ordered = Atoms(symbols=list(ls_syms_matched), positions=list(ls_cart_matched), cell=cell, pbc=True)

    # Build HS atoms (aligned positions -> P_aligned_cart)
    hs_syms_ordered = syms_hs_matched.copy()
    hs_symbols_for_cif = []
    for i, sym in enumerate(hs_syms_ordered):
        lab = sym + "_HS"
        if i == max_idx:
            lab = sym + "_MAX_HS"
        hs_symbols_for_cif.append(sym)  # keep actual chemical symbol (ASE expects real elements)

    atoms_hs_ordered = Atoms(symbols=list(hs_syms_ordered), positions=list(P_aligned_cart), cell=cell, pbc=True)

    # Attach labels arrays so CIF writer puts them into _atom_site_label
    labels_ls = [f"{sym}_LS" for sym in ls_syms_matched]
    labels_hs = []
    for i, sym in enumerate(hs_syms_ordered):
        if i == max_idx:
            labels_hs.append(f"{sym}_MAX_HS")
        else:
            labels_hs.append(f"{sym}_HS")
    atoms_ls_ordered.new_array("labels", np.array(labels_ls, dtype=object))
    atoms_hs_ordered.new_array("labels", np.array(labels_hs, dtype=object))

    combined = atoms_ls_ordered + atoms_hs_ordered
    combined.set_cell(cell)
    combined.wrap()

    write(cif_out, combined)
    print(f"Wrote overlay CIF for VESTA -> {cif_out}")
    print("In VESTA, open the CIF, go to Edit->Atoms and check atom labels (they contain _LS, _HS, and _MAX_HS).")
    print("\nDone.")

if __name__ == "__main__":
    main()


Unique sites found: HS=204  LS=204

=== RESULTS ===
Number of atoms compared: 204
Average RMSD (Å): 0.2686
Maximum deviation (Å): 1.1400  (matched atom index: 27, element: S)
Wrote per-atom text -> rmsd_out_per_atom.txt
Wrote per-atom CSV  -> rmsd_out_per_atom.csv
Wrote overlay CIF for VESTA -> rmsd_out_overlay.cif
In VESTA, open the CIF, go to Edit->Atoms and check atom labels (they contain _LS, _HS, and _MAX_HS).

Done.
